In [33]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
artists = pd.read_csv('data_by_artist.csv')
artists

,artist_name,artist_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity,count
0,Frank Sinatra,792507,0.384478,0.238017,0.364288,110.181698,-14.271141,1,5,0.735648,0.020855,0.232106,0.049614,189179.9255,26.004383,1369
1,Vladimir Horowitz,119107,0.343210,0.118844,0.225951,94.900679,-23.193418,1,1,0.990070,0.879508,0.183812,0.043360,266541.1251,3.592378,1207
2,Johnny Cash,816890,0.619803,0.449381,0.680662,115.037747,-11.593104,1,10,0.685637,0.022647,0.242243,0.098216,162279.2672,26.614130,1104
3,Billie Holiday,79016,0.572637,0.201368,0.498934,109.912172,-13.225966,1,5,0.908499,0.013064,0.217727,0.062432,185131.4530,15.621005,1095
4,Bob Dylan,66915,0.512598,0.477932,0.551934,126.160149,-11.184330,1,7,0.562567,0.034211,0.308978,0.064535,256713.4203,30.860806,1092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5849,Natalie La Rose,3359519,0.830000,0.520000,0.735000,104.990000,-8.714000,1,0,0.000792,0.000013,0.065600,0.037600,189907.0000,64.000000,1
5850,Sarah Ross,3381566,0.721000,0.944000,0.626000,85.002000,-5.982000,1,8,0.013000,0.000000,0.320000,0.159000,262760.0000,52.000000,1
5851,Rotimi,3410250,0.637000,0.501000,0.431000,103.993000,-6.148000,0,0,0.229000,0.000059,0.099000,0.187000,185461.0000,71.000000,1
5852,Jillian Jacqueline,3455945,0.547000,0.672000,0.283000,155.791000,-5.023000,1,11,0.304000,0.000000,0.099600,0.049600,213133.0000,58.000000,1


In [14]:
years = pd.read_csv('data_by_year.csv')
years

,year,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,duration_ms,popularity
0,1921,0.425661,0.236784,0.425495,100.397758,-17.095437,1,7,0.895823,0.322330,0.215814,0.077258,229911.9141,0.351562
1,1922,0.480000,0.237026,0.534056,101.376139,-19.179958,1,10,0.939236,0.440470,0.238647,0.115419,167904.5417,0.138889
2,1923,0.568462,0.246936,0.624788,112.456598,-14.373882,1,0,0.976329,0.401932,0.236656,0.098619,178356.3018,5.727811
3,1924,0.548654,0.347033,0.668574,120.653359,-14.202304,1,10,0.935575,0.583955,0.237875,0.090210,188461.6498,0.603376
4,1925,0.571890,0.264373,0.616430,115.671715,-14.516707,1,5,0.965422,0.408893,0.243094,0.115457,184130.6996,2.707224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2016,0.599976,0.592877,0.430769,119.070344,-7.949913,1,0,0.280290,0.074646,0.180198,0.107298,219400.7638,61.371254
96,2017,0.612286,0.586739,0.414465,116.840277,-8.422697,1,1,0.289916,0.098209,0.194218,0.111752,209343.6130,64.861500
97,2018,0.664930,0.590591,0.447141,122.004325,-7.253666,1,1,0.271941,0.035948,0.171781,0.128140,200919.1190,67.276000
98,2019,0.644215,0.578796,0.465856,118.868163,-8.041738,1,1,0.289298,0.076518,0.167161,0.124799,197733.1330,69.655500


In [15]:
full = pd.read_csv('full_music_data.csv')
full

,artist_names,artists_id,danceability,energy,valence,tempo,loudness,mode,key,acousticness,instrumentalness,liveness,speechiness,explicit,duration_ms,popularity,year,release_date,song_title (censored)
0,"[""Fat Freddy's Drop""]",[178301],0.600,0.365,0.131,130.046,-13.083,0,9,0.06720,0.585000,0.0921,0.0498,0,437200,54,2005,2005,Ernie
1,"[""Fat Freddy's Drop""]",[178301],0.874,0.326,0.179,119.620,-13.302,0,11,0.01360,0.148000,0.0993,0.1310,0,581008,53,2005,2005,Wandering Eye
2,"[""Fat Freddy's Drop""]",[178301],0.670,0.531,0.336,139.385,-8.267,0,9,0.01560,0.345000,0.3060,0.0377,0,431293,55,2009,8/7/2009,The Raft
3,"[""Alexander O'Neal""]",[625201],0.761,0.702,0.850,104.773,-8.523,1,7,0.10800,0.000031,0.0935,0.0389,0,304427,34,1991,1/1/1991,All True Man
4,"[""Alexander O'Neal""]",[625201],0.661,0.828,0.902,115.078,-12.673,0,11,0.27600,0.000001,0.2870,0.0390,0,264933,37,1987,7/29/1987,(What Can I Say) To Make You Love Me
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98335,['ZZ Top'],[690254],0.276,0.892,0.715,80.475,-7.035,1,11,0.40900,0.000000,0.7140,0.0893,0,115973,33,1975,4/18/1975,Jailhouse Rock - **** Remaster
98336,['ZZ Top'],[690254],0.700,0.592,0.906,109.847,-10.434,1,11,0.10800,0.018400,0.1180,0.0681,0,263627,32,1976,11/29/1976,It's Only Love
98337,['ZZ Top'],[690254],0.709,0.709,0.863,111.544,-12.023,1,2,0.11800,0.000011,0.1250,0.0370,0,158400,31,1987,1987,Balinese
98338,['ZZ Top'],[690254],0.552,0.651,0.533,161.548,-10.624,0,4,0.00494,0.034900,0.1320,0.2290,0,232533,43,1992,4/13/1992,La Grange - **** Remaster


In [16]:
influence = pd.read_csv('influence_data.csv')
influence

,influencer_id,influencer_name,influencer_main_genre,influencer_active_start,follower_id,follower_name,follower_main_genre,follower_active_start
0,759491,The Exploited,Pop/Rock,1980,74,Special Duties,Pop/Rock,1980
1,25462,Tricky,Electronic,1990,335,PJ Harvey,Pop/Rock,1990
2,66915,Bob Dylan,Pop/Rock,1960,335,PJ Harvey,Pop/Rock,1990
3,71209,Leonard Cohen,Pop/Rock,1950,335,PJ Harvey,Pop/Rock,1990
4,91438,The Gun Club,Pop/Rock,1980,335,PJ Harvey,Pop/Rock,1990
...,...,...,...,...,...,...,...,...
42765,580300,Sufjan Stevens,Pop/Rock,1990,3661738,Rosemary & Garlic,Pop/Rock,2010
42766,261309,Vybz Kartel,Reggae,2000,3670556,Trinidad Cardona,R&B;,2010
42767,467203,Michael Jackson,R&B;,1960,3670556,Trinidad Cardona,R&B;,2010
42768,2518003,Popcaan,Reggae,2000,3670556,Trinidad Cardona,R&B;,2010


In [18]:
len(influence)

42770

In [57]:
influenceGraph = nx.DiGraph()

In [ ]:
i = 0
while i < len(influence):
    influencer = artists[artists['artist_name'] == influence['influencer_name'][i]]
    influencer = influencer[['energy','valence', 'key', 'acousticness', 'instrumentalness', 'speechiness']]
    v1 = influencer.to_numpy()
    follower = artists[artists['artist_name'] == influence['follower_name'][i]]
    follower = follower[['energy','valence', 'key', 'acousticness', 'instrumentalness', 'speechiness']]
    v2 = follower.to_numpy()
    print(v1)
    print(v2)
    weight = cosine_similarity(v1, v2) * 1000
    print(weight)
    influenceGraph.add_edge(influence['influencer_name'][i], influence['follower_name'][i], weight = weight)
    i += 1

[[8.59571429e-01 4.19285714e-01 7.00000000e+00 3.69747140e-02
  4.04277900e-03 1.52171429e-01]]
[[8.65e-01 4.87e-01 9.00e+00 2.64e-02 1.85e-03 4.60e-02]]
[[999.49996324]]
[[0.63083333 0.55633333 0.         0.22453833 0.18496888 0.13008333]]
[[0.49487879 0.49157576 0.         0.18777163 0.14303846 0.03791818]]
[[993.96118588]]
[[0.47793205 0.5519337  7.         0.56256679 0.03421143 0.06453489]]
[[0.49487879 0.49157576 0.         0.18777163 0.14303846 0.03791818]]
[[119.24012292]]
[[0.25280685 0.33891781 9.         0.7297452  0.02733073 0.03877945]]
[[0.49487879 0.49157576 0.         0.18777163 0.14303846 0.03791818]]
[[65.12950161]]
[[0.80958333 0.547      9.         0.1136745  0.04136167 0.07655833]]
[[0.49487879 0.49157576 0.         0.18777163 0.14303846 0.03791818]]
[[104.80671488]]
[[5.67333333e-01 8.94333333e-01 5.00000000e+00 1.18533333e-01
  6.59333000e-04 6.27666670e-02]]
[[0.49487879 0.49157576 0.         0.18777163 0.14303846 0.03791818]]
[[197.64979263]]
[[0.463  0.578  9. 

[[931.99897876]]
[[7.2500000e-01 7.0000000e-01 9.0000000e+00 6.7400000e-02 3.5700000e-07
  8.2466667e-02]]
[[0.82040323 0.40042258 9.         0.01723142 0.17434948 0.05254193]]
[[999.19106012]]
[[0.66586667 0.36048667 2.         0.08857801 0.16252333 0.05264   ]]
[[0.82040323 0.40042258 9.         0.01723142 0.17434948 0.05254193]]
[[963.7399528]]
[[0.71646075 0.53921028 8.         0.21113931 0.17333124 0.09865981]]
[[0.82040323 0.40042258 9.         0.01723142 0.17434948 0.05254193]]
[[999.42134901]]
[[0.39708878 0.4946415  7.         0.46080888 0.04617556 0.03768605]]
[[0.82040323 0.40042258 9.         0.01723142 0.17434948 0.05254193]]
[[996.97626238]]
[[0.58708   0.69528   0.        0.1990712 0.1112598 0.084332 ]]
[[0.82040323 0.40042258 9.         0.01723142 0.17434948 0.05254193]]
[[92.38048631]]
[[0.90390756 0.33906177 4.         0.02006071 0.22803874 0.09990336]]
[[0.82040323 0.40042258 9.         0.01723142 0.17434948 0.05254193]]
[[989.8887884]]
[[0.79480952 0.56952381 9.    

[[0.34522154 0.58       9.         0.54480308 0.02934325 0.07171846]]
[[0.47179167 0.62066667 7.         0.5947375  0.29353897 0.04265   ]]
[[998.28129618]]
[[0.30053    0.44124    7.         0.54885    0.44933071 0.04484167]]
[[0.47179167 0.62066667 7.         0.5947375  0.29353897 0.04265   ]]
[[999.12289923]]
[[0.54682697 0.61965954 0.         0.36035593 0.05190613 0.05339502]]
[[0.47179167 0.62066667 7.         0.5947375  0.29353897 0.04265   ]]
[[136.62752802]]
[[0.21066667 0.43933333 2.         0.93066667 0.93       0.05376667]]
[[0.47179167 0.62066667 7.         0.5947375  0.29353897 0.04265   ]]
[[879.29131147]]
[[0.25935519 0.45916132 2.         0.72938679 0.28788264 0.04763774]]
[[0.47179167 0.62066667 7.         0.5947375  0.29353897 0.04265   ]]
[[953.68535953]]
[[0.21763793 0.47844828 5.         0.83031035 0.25269384 0.05627069]]
[[0.47179167 0.62066667 7.         0.5947375  0.29353897 0.04265   ]]
[[996.50879017]]
[[0.23801673 0.36428824 5.         0.73564823 0.02085498 0

[[7.55e-01 8.24e-01 9.00e+00 1.57e-02 9.22e-06 4.53e-02]]
[[109.12941154]]
[[ 0.49138889  0.84563889 10.          0.46273611  0.12264449  0.06648333]]
[[7.55e-01 8.24e-01 9.00e+00 1.57e-02 9.22e-06 4.53e-02]]
[[998.32826303]]
[[ 0.69   0.79  11.     0.527  0.501  0.345]]
[[7.55e-01 8.24e-01 9.00e+00 1.57e-02 9.22e-06 4.53e-02]]
[[997.17429304]]
[[0.65918672 0.71942739 2.         0.1238621  0.13065461 0.04532822]]
[[0.7705   0.8425   6.       0.0254   0.149919 0.05115 ]]
[[963.0289685]]
[[0.84930769 0.48258077 9.         0.13335411 0.26677545 0.08112692]]
[[0.7705   0.8425   6.       0.0254   0.149919 0.05115 ]]
[[995.76197853]]
[[0.66637113 0.6502701  7.         0.220512   0.03572722 0.04817784]]
[[0.7705   0.8425   6.       0.0254   0.149919 0.05115 ]]
[[997.84822808]]
[[6.26e-01 6.55e-01 1.10e+01 2.46e-02 6.08e-03 1.02e-01]]
[[0.7705   0.8425   6.       0.0254   0.149919 0.05115 ]]
[[994.11122755]]
[[0.52584615 0.66461538 8.         0.47897608 0.21705387 0.05142308]]
[[0.7705   0.842

[[195.09342541]]
[[0.54973933 0.624882   1.         0.19907555 0.06265852 0.07452233]
 [0.54973933 0.624882   1.         0.19907555 0.06265852 0.07452233]
 [0.54973933 0.624882   1.         0.19907555 0.06265852 0.07452233]]
[[0.57081333 0.43770667 4.         0.37959733 0.00712678 0.10803467]]
[[866.83399452]
 [866.83399452]
 [866.83399452]]
[[ 0.62855814  0.5981938  11.          0.20131359  0.01354304  0.12008295]]
[[0.57081333 0.43770667 4.         0.37959733 0.00712678 0.10803467]]
[[992.1054978]]
[[0.3314846  0.45450404 9.         0.77993828 0.03847298 0.05817575]]
[[0.57081333 0.43770667 4.         0.37959733 0.00712678 0.10803467]]
[[992.71946607]]
[[3.9375e-01 7.2350e-01 5.0000e+00 5.0425e-01 4.3500e-05 3.2175e-02]]
[[0.57081333 0.43770667 4.         0.37959733 0.00712678 0.10803467]]
[[997.2152386]]
[[3.7560e-01 4.5440e-01 0.0000e+00 5.1360e-01 1.8842e-04 4.4540e-02]]
[[0.57081333 0.43770667 4.         0.37959733 0.00712678 0.10803467]]
[[191.71336257]]
[[0.52565635 0.61599448 

...

[[0.32885833 0.43034444 2.         0.69113889 0.02672924 0.08020278]]
[[9.470e-02 3.705e-01 1.100e+01 8.600e-01 2.230e-05 5.295e-02]]
[[944.31560488]]
[[0.20298636 0.34503333 9.         0.83375758 0.01751032 0.04454697]]
[[9.470e-02 3.705e-01 1.100e+01 8.600e-01 2.230e-05 5.295e-02]]
[[999.78871854]]
[[6.49166667e-01 4.55000000e-01 1.00000000e+01 5.82166667e-01
  1.25167000e-04 5.10500000e-01]]
[[9.41857143e-01 7.38285710e-02 1.00000000e+00 8.18857143e-01
  1.13000000e-06 9.12857143e-01]]
[[624.94279045]]
[[8.234e-01 3.134e-01 1.100e+01 9.364e-01 4.420e-05 9.052e-01]]
[[9.41857143e-01 7.38285710e-02 1.00000000e+00 8.18857143e-01
  1.13000000e-06 9.12857143e-01]]
[[653.94983289]]
[[4.48448276e-01 7.18689655e-01 5.00000000e+00 6.32417241e-01
  5.24000000e-05 4.44551720e-02]]
[[ 0.8084      0.69383333 10.          0.29013     0.09419568  0.13472333]]
[[992.72576498]]
[[ 0.55366667  0.87622222 10.          0.46277778  0.49485433  0.03853333]]
[[ 0.8084      0.69383333 10.          0.29013 

[[0.50571429 0.42228571 2.         0.25554021 0.02205556 0.03408571]]
[[8.07575758e-01 5.12800000e-01 4.00000000e+00 3.88263940e-02
  2.71931400e-03 5.84787880e-02]]
[[989.69752092]]
[[0.69296154 0.63909615 7.         0.18190579 0.06434752 0.04165769]]
[[8.07575758e-01 5.12800000e-01 4.00000000e+00 3.88263940e-02
  2.71931400e-03 5.84787880e-02]]
[[994.21781478]]
[[0.6737     0.6685     2.         0.36194333 0.10780848 0.05169667]]
[[8.07575758e-01 5.12800000e-01 4.00000000e+00 3.88263940e-02
  2.71931400e-03 5.84787880e-02]]
[[963.65013186]]
[[0.77986463 0.51890044 9.         0.16506723 0.0260743  0.06741485]]
[[8.07575758e-01 5.12800000e-01 4.00000000e+00 3.88263940e-02
  2.71931400e-03 5.84787880e-02]]
[[991.35969608]]
[[0.6094382  0.65666292 7.         0.28478135 0.04028922 0.03492472]]
[[8.07575758e-01 5.12800000e-01 4.00000000e+00 3.88263940e-02
  2.71931400e-03 5.84787880e-02]]
[[992.73677474]]
[[0.61816667 0.55958333 2.         0.18662519 0.32627339 0.04838889]]
[[8.07575758e-0

  1.99202315e-01 1.22325000e-01]]
[[9.397000e-01 4.118150e-01 8.000000e+00 1.806660e-03 2.028309e-02
  1.482900e-01]]
[[943.08628507]]
[[0.83006484 0.29160968 8.         0.01246648 0.26623775 0.0845129 ]]
[[9.397000e-01 4.118150e-01 8.000000e+00 1.806660e-03 2.028309e-02
  1.482900e-01]]
[[999.29996887]]
[[7.41000000e-01 5.33333333e-01 9.00000000e+00 4.40000000e-03
  3.77343300e-03 8.95000000e-02]]
[[9.397000e-01 4.118150e-01 8.000000e+00 1.806660e-03 2.028309e-02
  1.482900e-01]]
[[999.32770384]]
[[4.009375e-01 3.919250e-01 7.000000e+00 6.391250e-01 3.115102e-03
  4.955625e-02]]
[[2.17833333e-01 2.48714286e-01 1.00000000e+01 9.10095238e-01
  8.48553000e-04 6.95238100e-02]]
[[998.90269065]]
[[3.39366667e-01 6.14433333e-01 0.00000000e+00 8.47566667e-01
  9.95000000e-05 8.93666670e-02]]
[[2.17833333e-01 2.48714286e-01 1.00000000e+01 9.10095238e-01
  8.48553000e-04 6.95238100e-02]]
[[90.53682617]]
[[0.27967273 0.4818303  0.         0.81642424 0.00386168 0.05177879]]
[[2.17833333e-01 2.487

[[0.30822857 0.41786018 0.         0.65571081 0.205073   0.05457107]]
[[0.517125   0.76225    5.         0.10555    0.01469651 0.0369875 ]]
[[125.82600348]]
[[0.32841852 0.42137037 9.         0.68038519 0.46397428 0.04372593]]
[[0.517125   0.76225    5.         0.10555    0.01469651 0.0369875 ]]
[[989.80923076]]
[[0.56325    0.478875   0.         0.13023495 0.29670097 0.0464125 ]]
[[0.832      0.62973913 0.         0.07712824 0.12113367 0.04402609]]
[[959.94769506]]
[[0.56474667 0.37555697 7.         0.25449917 0.10525787 0.04692   ]]
[[0.832      0.62973913 0.         0.07712824 0.12113367 0.04402609]]
[[99.8424244]]
[[0.755    0.7245   7.       0.050325 0.10935  0.03585 ]]
[[0.832      0.62973913 0.         0.07712824 0.12113367 0.04402609]]
[[147.81813738]]
[[0.59235659 0.57925892 2.         0.42715814 0.12266223 0.06221783]]
[[0.832      0.62973913 0.         0.07712824 0.12113367 0.04402609]]
[[389.65636326]]
[[0.76157143 0.39674286 9.         0.08073429 0.80342857 0.03678571]]
[[

[[0.87146512 0.37291861 8.         0.11975931 0.19856847 0.0878814 ]]
[[0.64932258 0.55367742 7.         0.32652581 0.0125148  0.03772581]]
[[998.58231436]]
[[0.5725    0.4665    9.        0.3931    0.0936635 0.04795  ]]
[[0.64932258 0.55367742 7.         0.32652581 0.0125148  0.03772581]]
[[999.18101933]]
[[0.79047727 0.5363053  9.         0.07851088 0.03324141 0.04770303]]
[[0.64932258 0.55367742 7.         0.32652581 0.0125148  0.03772581]]
[[999.09355739]]
[[0.62292063 0.61961111 2.         0.331065   0.28413833 0.0440627 ]]
[[0.64932258 0.55367742 7.         0.32652581 0.0125148  0.03772581]]
[[944.81082222]]
[[0.54682697 0.61965954 0.         0.36035593 0.05190613 0.05339502]]
[[0.64932258 0.55367742 7.         0.32652581 0.0125148  0.03772581]]
[[128.1611755]]
[[7.18333333e-01 3.70333333e-01 0.00000000e+00 5.15526700e-03
  2.52000000e-05 4.38166670e-02]]
[[0.64932258 0.55367742 7.         0.32652581 0.0125148  0.03772581]]
[[118.1019389]]
[[0.75353243 0.39114595 9.         0.123

In [56]:
influenceGraph['Bob Dylan']['Steely Dan']

{'weight': array([[950.34487452]])}

In [116]:
idDict = {}
for i in range(len(artists)):
    idDict[artists['artist_id'][i]] = artists['artist_name'][i]
    artistDict[artists['artist_name'][i]] = artists['artist_id'][i]

influence = pd.read_csv('influence_data.csv')
iGraph = nx.DiGraph()
for artist in artists['artist_id']:
    iGraph.add_node(artist, weight = 0)
    
for i in range(len(influence)):
    influencer = influence["influencer_id"][i]
    follower = influence['follower_id'][i]
    
    iGraph.add_edge(influencer,follower)    

arts = []
artnames = []
scores = []
for node in iGraph.nodes():
    children = list(iGraph.successors(node))
    total = 0
    for child in children:
        total += iGraph.out_degree[child]
        
    iGraph.nodes[node]['weight'] = total
    arts.append(node)
    try:
        artnames.append(idDict[node])
    except KeyError:
        artnames.append('Unknown')
    scores.append(total)
    
   
a = pd.DataFrame({'artists':arts, 'names':artnames, 'scores':scores})
a = a.sort_values(by=['scores'], ascending = False)
print(a[:20])

      artists                   names  scores
15     754032             The Beatles   11237
4       66915               Bob Dylan    7928
5      894465      The Rolling Stones    6341
137    120521             Chuck Berry    5897
7      180228           Elvis Presley    4749
210    840402  The Velvet Underground    4380
176    824022          Little Richard    4371
147    354105            Jimi Hendrix    4312
201    549797           Hank Williams    4134
35     100160               The Kinks    4067
24     577627                 The Who    4025
33     531986             David Bowie    3727
158    128099             James Brown    3725
390    538677             Buddy Holly    3502
1258   418740             Sex Pistols    3317
623    562304             The Stooges    3240
185    608701            Muddy Waters    3228
459     55128              Bo Diddley    3169
17     139026            Led Zeppelin    3143
6       41874          The Beach Boys    2955
